In [1]:
%reload_ext autoreload
%autoreload

from sympy import *
from Electromagnetics.EMEqs_sym import *
from IPython.display import display, Math

maxwell = MaxwellEqs_r()

MF = maxwell.getMaxwellFaraday()
MA = maxwell.getMaxwellAmper(sigma=True)

MF = MF[0] - MF[1]
MA = MA[0] - MA[1]

display(Math(latex(MF)))
display(Math(latex(MA)))

Hz_rep = solve(MF[2], maxwell.Hz)[0]
Ez_rep = solve(MA[2], maxwell.Ez)[0]

display(Math('H_z = ' + latex(Hz_rep)))
display(Math('E_z = ' + latex(Ez_rep)))


MF = Matrix([MF[0].subs(maxwell.Ez, Ez_rep), MF[1].subs(maxwell.Ez, Ez_rep)])
MA = Matrix([MA[0].subs(maxwell.Hz, Hz_rep), MA[1].subs(maxwell.Hz, Hz_rep)])

MFz = maxwell.getZPropEqs(MF)
MAz = maxwell.getZPropEqs(MA)

display(Math(latex(MFz)))
display(Math(latex(MAz)))


for i in range(MFz.shape[0]):
    for j in range(MFz.shape[1]):
        MFz[i,j] = (MFz[i,j].doit()*exp(I*maxwell.k*maxwell.z)).simplify()
for i in range(MAz.shape[0]):
    for j in range(MAz.shape[1]):
        MAz[i,j] = (MAz[i,j].doit()*exp(I*maxwell.k*maxwell.z)).simplify()

## MF:maxwell faraday -- MA:maxwell ampere        
display(Math('MF = ' + latex(MFz)))
display(Math('MA = ' + latex(MAz)))

MF.shape


(2, 1)

In [2]:
%reload_ext autoreload
%autoreload

from Electromagnetics.EMEqs_sym import *
from IPython.display import display, Math
from sympy import *

maxwell = MaxwellEqs_r()

MF = maxwell.getMaxwellFaraday()
MA = maxwell.getMaxwellAmper(sigma=True)

MF = MF[0] - MF[1]
MA = MA[0] - MA[1]

display(Math(latex(MF)))
display(Math(latex(MA)))


MFz = maxwell.getZPropEqs(MF)
MAz = maxwell.getZPropEqs(MA)

display(Math(latex(MFz)))
display(Math(latex(MAz)))


for i in range(MFz.shape[0]):
    for j in range(MFz.shape[1]):
        MFz[i,j] = (MFz[i,j].doit()*exp(I*maxwell.k*maxwell.z)).simplify()
for i in range(MAz.shape[0]):
    for j in range(MAz.shape[1]):
        MAz[i,j] = (MAz[i,j].doit()*exp(I*maxwell.k*maxwell.z)).simplify()

## MF:maxwell faraday -- MA:maxwell ampere        
display(Math('MF = ' + latex(MFz.T)))
display(Math('MA = ' + latex(MAz.T)))

MFz = maxwell.ZPropEqs_ReplFunBySym(MFz)
MAz = maxwell.ZPropEqs_ReplFunBySym(MAz)
display(Math('MF = ' + latex(MFz.T)))
display(Math('MA = ' + latex(MAz.T)))

k_0 = Symbol('k_0')
print('normalizing to k_0')
MFz[0] = (MFz[0]/(-I*k_0)).simplify()
MFz[1] = (MFz[1]/(I*k_0)).simplify()
MAz[0] = (MAz[0]/(-I*k_0)).simplify()
MAz[1] = (MAz[1]/(I*k_0)).simplify()
display(Math('MF = ' + latex(MFz.T)))
display(Math('MA = ' + latex(MAz.T)))


MFz = MFz.subs(maxwell.k, 0)
MAz = MAz.subs(maxwell.k, 0)
display(Math('MF = ' + latex(MFz.T)))
display(Math('MA = ' + latex(MAz.T)))

Eqs = [MFz[1], MFz[0], MAz[1], MAz[0], MAz[2], MFz[2]]
display(Math('Eqs = ' + latex(Matrix(Eqs))))


normalizing to k_0


In [3]:

from scipy import constants
import math
freq = 3.0*constants.tera
omega_0 = 2.0*math.pi*freq
eps_0 = constants.epsilon_0
mu_0 = constants.mu_0
eps_r = 1.0
mu_r = 1.0
k_0_val = omega_0*sqrt(mu_0*eps_0)

from Electromagnetics.graphene import *
import numpy as np 

mu_c = 0.3*constants.eV
B_0 = 0.0
tau = 0.2*constants.pico
T = 100.0

sigma_d, sigma_o = condKuboLorentzian(mu_c, B_0, tau, np.array([omega_0]), T)

sig_graph = sigma_d[0]
del_graph = 1000.0*constants.nano
width_graph = 100.0*constants.micro

Eqs1 = [Eqs[i].subs([(maxwell.mu, mu_0), (maxwell.eps, eps_0), (maxwell.sigma, 0.0), (maxwell.omega, omega_0), (k_0, k_0_val)]).simplify() for i in range(len(Eqs))]
display(Math('Eqs1 = ' + latex(Matrix(Eqs1))))

print(srepr(Eqs1[0]))

Eqs2 = [Eqs[i].subs([(maxwell.mu, mu_r*mu_0), (maxwell.eps, eps_r*eps_0), (maxwell.sigma, sig_graph/del_graph), (maxwell.omega, omega_0), (k_0, k_0_val)]).simplify() for i in range(len(Eqs))]
display(Math('Eqs2 = ' + latex(Matrix(Eqs2))))


var_list = [maxwell.Ex, maxwell.Ey, maxwell.Hx, maxwell.Hy, maxwell.Ez, maxwell.Hz]
display(Math('Vars = ' + latex(Matrix(var_list))))



Add(Mul(Float('376.73031346177066', prec=15), Symbol('H_{y}')), Mul(Float('1.5904483864123141e-5', prec=15), I, Derivative(Symbol('E_{z}'), Symbol('x'))))


In [4]:
print('disintegrating equations')
Eqs_b = [[Eqs[0].subs(maxwell.Ez, 0).doit().simplify(), Eqs[0].subs(maxwell.Hy, 0).simplify()],
             [Eqs[1].subs(maxwell.Ez, 0).doit().simplify(), Eqs[1].subs(maxwell.Hx, 0).simplify()],
             [Eqs[2].subs(maxwell.Hz, 0).doit().simplify(), Eqs[2].subs(maxwell.Ey, 0).simplify()],
             [Eqs[3].subs(maxwell.Hz, 0).doit().simplify(), Eqs[3].subs(maxwell.Ex, 0).simplify()],
             [Eqs[4].subs([(maxwell.Hx, 0), (maxwell.Hy, 0)]).doit().simplify(), Eqs[4].subs([(maxwell.Ez, 0), (maxwell.Hy, 0)]).subs(Derivative(0, maxwell.x), 0).simplify(), Eqs[4].subs([(maxwell.Ez, 0), (maxwell.Hx, 0)]).subs(Derivative(0, maxwell.y), 0).simplify()],
             [Eqs[5].subs([(maxwell.Ex, 0), (maxwell.Ey, 0)]).doit().simplify(), Eqs[5].subs([(maxwell.Hz, 0), (maxwell.Ey, 0)]).subs(Derivative(0, maxwell.x), 0).simplify(), Eqs[5].subs([(maxwell.Hz, 0), (maxwell.Ex, 0)]).subs(Derivative(0, maxwell.y), 0).simplify()]]
for i in range(len(Eqs_b)):
    display(Math('Eqs_b[{}] = '.format(i) + latex(Matrix(Eqs_b[i]).T)))

print('Eq1 boundary equations ')
Eqs1_b = [[Eqs_b[i][j].subs([(maxwell.mu, mu_0), (maxwell.eps, eps_0), (maxwell.sigma, 0.0), (maxwell.omega, omega_0), (k_0, k_0_val)]) for j in range(len(Eqs_b[i]))] for i in range(len(Eqs_b))]
for i in range(len(Eqs1_b)):
    display(Math('EQs1_b[{}] = '.format(i) + latex(Matrix(Eqs1_b[i]).T)))

print('Eq2 boundary equations ')
Eqs2_b = [[Eqs_b[i][j].subs([(maxwell.mu, mu_r*mu_0), (maxwell.eps, eps_r*eps_0), (maxwell.sigma, sig_graph/del_graph), (maxwell.omega, omega_0), (k_0, k_0_val)]) for j in range(len(Eqs_b[i]))] for i in range(len(Eqs_b))]
for i in range(len(Eqs2_b)):
    display(Math('Eqs2_b[{}] = '.format(i) + latex(Matrix(Eqs2_b[i]).T)))


disintegrating equations


Eq1 boundary equations 


Eq2 boundary equations 


In [5]:
%reload_ext autoreload
%autoreload

from Electromagnetics.RecGrid2D import RG2D
from Electromagnetics.Misc import *

W_x = 3.0*width_graph
W_y = 2.0*width_graph
rg = RG2D(0.0, W_x, 0.0, W_y, W_x/10.0, W_y/10.0)
#rg.RefineCells(list(range(len(rg.cells))))

w_edge_mode = 10.0*constants.micro
P00 = Point2D(W_x/2.0-width_graph/2.0-w_edge_mode, W_y/2.0-w_edge_mode)
P11 = Point2D(W_x/2.0-width_graph/2.0+w_edge_mode, W_y/2.0+del_graph+w_edge_mode)
rect_edge_l = [P00, P11]
P00 = Point2D(W_x/2.0+width_graph/2.0-w_edge_mode, W_y/2.0-w_edge_mode)
P11 = Point2D(W_x/2.0+width_graph/2.0+w_edge_mode, W_y/2.0+del_graph+w_edge_mode)
rect_edge_r = [P00, P11]
w_bulk_mode = 10.0*constants.micro
P00 = Point2D(W_x/2.0-width_graph/2.0-w_edge_mode, W_y/2.0-w_edge_mode)
P11 = Point2D(W_x/2.0+width_graph/2.0+w_edge_mode, W_y/2.0+del_graph+w_edge_mode)
rect_bulk = [P00, P11]
rg.RefineRectangle(rect_bulk)
rg.RefineRectangle(rect_bulk)
rg.RefineRectangle(rect_bulk)
rg.RefineRectangle(rect_edge_l)
rg.RefineRectangle(rect_edge_r)


P00 = Point2D(W_x/2.0-width_graph/2.0, W_y/2.0)
P11 = Point2D(W_x/2.0+width_graph/2.0, W_y/2.0+del_graph)
rect_graph = [P00, P11]
rg.RefineThinRectangle(rect_graph)
rg.RefineRectangleBorders(rect_graph)
#rg.RefineRectangleBorders(rect_graph)


print('n_cells: ', len(rg.cells), ' n_sides: ', len(rg.sides), ' n_nodes: ', len(rg.nodes))



##-------------
tic()

region_fs = "freespace"
rg.AddExpression(region_fs, Eqs1, var_list)
region_di = "dielectric"
rg.AddExpression(region_di, Eqs2, var_list)

eq_list1 = [{'zero':Float(0), 'one':(Eqs1_b[0][0]+Eqs1_b[0][1])/2.0, 'two':(Eqs1_b[0][0]+Eqs1_b[0][1])},
            {'zero':Float(0), 'one':(Eqs1_b[1][0]+Eqs1_b[1][1])/2.0, 'two':(Eqs1_b[1][0]+Eqs1_b[1][1])},
            {'zero':Float(0), 'one':(Eqs1_b[2][0]/2.0+Eqs1_b[2][1]), 'two':(Eqs1_b[2][0]+Eqs1_b[2][1])},
            {'zero':Float(0), 'one':(Eqs1_b[3][0]/2.0+Eqs1_b[3][1]), 'two':(Eqs1_b[3][0]+Eqs1_b[3][1])},
            {'zero':Float(0), 'one':(Eqs1_b[4][0]/4.0), 'two':(Eqs1_b[4][0]/2.0), 'two_x':(Eqs1_b[4][1]+Eqs1_b[4][2]/2.0), 'two_y':(Eqs1_b[4][1]/2.0+Eqs1_b[4][2]), 'three':(Eqs1_b[4][0]*3.0/4.0+Eqs1_b[4][1]+Eqs1_b[4][2]), 'four':(Eqs1_b[4][0]+Eqs1_b[4][1]+Eqs1_b[4][2])},
           {}]
var_type = ['SX', 'SY', 'SY', 'SX', 'N', 'C']
rg.SetRegionBoundaryEquations(region_fs, eq_list1, var_list, var_type)

eq_list2 = [{'zero':Float(0), 'one':(Eqs2_b[0][0]+Eqs2_b[0][1])/2.0, 'two':(Eqs2_b[0][0]+Eqs2_b[0][1])},
            {'zero':Float(0), 'one':(Eqs2_b[1][0]+Eqs2_b[1][1])/2.0, 'two':(Eqs2_b[1][0]+Eqs2_b[1][1])},
            {'zero':Float(0), 'one':(Eqs2_b[2][0]/2.0+Eqs2_b[2][1]), 'two':(Eqs2_b[2][0]+Eqs2_b[2][1])},
            {'zero':Float(0), 'one':(Eqs2_b[3][0]/2.0+Eqs2_b[3][1]), 'two':(Eqs2_b[3][0]+Eqs2_b[3][1])},
            {'zero':Float(0), 'one':(Eqs2_b[4][0]/4.0), 'two':(Eqs2_b[4][0]/2.0), 'two_x':(Eqs2_b[4][1]+Eqs2_b[4][2]/2.0), 'two_y':(Eqs2_b[4][1]/2.0+Eqs2_b[4][2]), 'three':(Eqs2_b[4][0]*3.0/4.0+Eqs2_b[4][1]+Eqs2_b[4][2]), 'four':(Eqs2_b[4][0]+Eqs2_b[4][1]+Eqs2_b[4][2])},
           {}]
var_type = ['SX', 'SY', 'SY', 'SX', 'N', 'C']
rg.SetRegionBoundaryEquations(region_di, eq_list2, var_list, var_type)

cells_fs = list(range(len(rg.cells)))
cells_di, nodes_di, sidesX_di, sidesY_di = rg.FindElementsMarkedByRectangle(rect_graph[0], rect_graph[1])
cells_fs, cells_di = rg.RegionsRemoveSharedCells([cells_fs, cells_di])
nodes_fs, sidesX_fs, sidesY_fs = rg.Cells_GetNodesSide(cells_fs)

rg.AssignVariableToSides(maxwell.Ex, sidesX_fs, region_fs, orient='X')
rg.AssignVariableToSides(maxwell.Ey, sidesY_fs, region_fs, orient='Y')
rg.AssignVariableToSides(maxwell.Hx, sidesY_fs, region_fs, orient='Y')
rg.AssignVariableToSides(maxwell.Hy, sidesX_fs, region_fs, orient='X')
rg.AssignVariableToNodes(maxwell.Ez, nodes_fs, region_fs)
rg.AssignVariableToCells(maxwell.Hz, cells_fs, region_fs)

rg.AssignVariableToSides(maxwell.Ex, sidesX_di, region_di, orient='X')
rg.AssignVariableToSides(maxwell.Ey, sidesY_di, region_di, orient='Y')
rg.AssignVariableToSides(maxwell.Hx, sidesY_di, region_di, orient='Y')
rg.AssignVariableToSides(maxwell.Hy, sidesX_di, region_di, orient='X')
rg.AssignVariableToNodes(maxwell.Ez, nodes_di, region_di)
rg.AssignVariableToCells(maxwell.Hz, cells_di, region_di)

rg.SetTotalIndicesForVariables()

print('grid: ', toc())
print('rg.CellsToIndexTotalShared: ', len(rg.CellsToIndexTotalShared))
print('rg.SidesToIndexTotalShared: ', len(rg.SidesToIndexTotalShared))
print('rg.NodesToIndexTotalShared: ', len(rg.NodesToIndexTotalShared))

A_coo = rg.ConstructInitialMatrix(save=True)
print("initial matrix", A_coo.nnz, toc())

A_coo = rg.ConstructInitialMatrix_Shared(A_coo, save=True)
print("initial matrix shared", A_coo.nnz, toc())

A_coo = rg.ApplyDirichletBoundaryCondition(A_coo, save=True)
print("initial matrix boundary conditions: ", A_coo.nnz, toc()) 

tic()
print("eliminating z components")
A_fin = rg.EliminateVarible_Direct(A_coo, [maxwell.Ez, maxwell.Hz])
print(toc())


tic()
print("solving eigenvalues")
k_eigs, k_vecs = rg.SolveEigenvalues(A_fin, 5, lambda_0=15.0)
print(toc())

print(k_eigs)
print(k_vecs.shape)

print("A_reduced nnz: ", A_fin.nnz)
print(toc())

cells_in_extended:  [69, 70, 71, 72, 73, 47, 48, 49, 50, 51, 58, 59, 60, 61, 62]
cells_in_extended:  [46, 52, 57, 63, 68, 74, 128, 131, 134, 136, 138, 139, 141, 142, 144, 145, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 58, 59, 60, 61, 62, 69, 70, 71, 72, 73, 122, 125]
cells_in_extended:  [256, 257, 258, 259, 260, 261, 262, 265, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 284, 287, 290, 293, 296, 299, 302, 59, 60, 61, 62, 70, 71, 72, 73, 122, 125, 128, 131, 134, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 185, 188, 195, 196, 198, 199, 201, 202, 204, 205, 207, 208, 210, 211, 213, 214, 216, 217, 219, 220, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]
cells_in_extended:  [179, 173, 174, 168, 151, 268, 58, 193, 69, 223, 386, 555, 537, 391, 486, 269, 270, 271, 536, 532, 533, 534, 53

[autoreload of Electromagnetics.graphene failed: Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 229, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'SymExprTree'
]
[autoreload of Electromagnetics.Misc failed: Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 229, in check
    superreload(m, reload, self.old_objects)
RuntimeError: maximum recursion depth exceeded in __instancecheck__
]


In [6]:
import numpy as np

tic()
rg.GetFromSolutionIndexToInitialIndex(k_vecs[:,0], A_coo)
print('GetFromSolutionIndexToInitialIndex : ', toc())

tic()
exy_cell = rg.InterpolateVariablesValuesToCellCenters([maxwell.Ex, maxwell.Ey])
ex = np.array(exy_cell[0])
ey = np.array(exy_cell[1])
print('InterpolateVariablesValuesToCellCenters : ', toc())


tic()
ex_sq = rg.CalculateSurfaceIntegral(ex, np.conjugate(ex))
ey_sq = rg.CalculateSurfaceIntegral(ey, np.conjugate(ey))
exy_len = np.sqrt(ex_sq + ey_sq)
ex /= exy_len
ey /= exy_len
print('exy_len: ', exy_len)
print('CalculateSurfaceIntegral : ', toc())



exy_max = GetMaxComplexVec2D(ex, ey)

print(ex.shape, ey.shape)
print(exy_max.shape)

ex_prev = ex
ey_prev = ey


GetFromSolutionIndexToInitialIndex :  0:00:06
InterpolateVariablesValuesToCellCenters :  0:00:00
exy_len:  (5.49932724557e-07+0j)
CalculateSurfaceIntegral :  0:00:00
(2554,) (2554,)
(2554,)


/usr/lib/python3/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:339: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [7]:
%reload_ext autoreload
%autoreload


from Electromagnetics.RecGrid2D import *
import queue

que = queue.Queue()
print("queue started..")
window = GUIMaker2D(wx=3.0, wy=6.0, wxp=600, wyp=400, que=que)
    
print('window started...')

window.AttachGrid(rg)
que.put(["updategrid"])
que.put(["toggleTestMode"])

queue started..
window started...


[autoreload of Electromagnetics.graphene failed: Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 229, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'SymExprTree'
]
[autoreload of Electromagnetics.Misc failed: Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 229, in check
    superreload(m, reload, self.old_objects)
RuntimeError: maximum recursion depth exceeded in __instancecheck__
]


In [8]:
import numpy as np
#from Electromagnetics import Misc 
from Electromagnetics.Misc import *

colors = GetColorMap(NormalizeArrayTo0n1(exy_max, logscale=True))
#print(colors)
que.put(["ColorCells", colors])

In [35]:
que.put(["MarkGridSides", sidesX_di, "red"])
que.put(["MarkGridSides", sidesY_di, "red"])

In [9]:

vbose = False
freq = 3.1*constants.tera

omega_0 = 2.0*math.pi*freq
k_0_val = omega_0*sqrt(mu_0*eps_0)

sigma_d, sigma_o = condKuboLorentzian(mu_c, B_0, tau, np.array([omega_0]), T)

sig_graph = sigma_d[0]
del_graph = 1.0*constants.micro
width_graph = 100.0*constants.micro

Eqs1 = [Eqs[i].subs([(maxwell.mu, mu_0), (maxwell.eps, eps_0), (maxwell.sigma, 0.0), (maxwell.omega, omega_0), (k_0, k_0_val)]).simplify() for i in range(len(Eqs))]
if vbose:
    display(Math('Eqs1 = ' + latex(Matrix(Eqs1))))
    print(srepr(Eqs1[0]))

Eqs2 = [Eqs[i].subs([(maxwell.mu, mu_r*mu_0), (maxwell.eps, eps_r*eps_0), (maxwell.sigma, sig_graph/del_graph), (maxwell.omega, omega_0), (k_0, k_0_val)]).simplify() for i in range(len(Eqs))]
if vbose:
    display(Math('Eqs2 = ' + latex(Matrix(Eqs2))))


var_list = [maxwell.Ex, maxwell.Ey, maxwell.Hx, maxwell.Hy, maxwell.Ez, maxwell.Hz]
if vbose:
    display(Math('Vars = ' + latex(Matrix(var_list))))



if vbose:
    print('disintegrating equations')
Eqs_b = [[Eqs[0].subs(maxwell.Ez, 0).doit().simplify(), Eqs[0].subs(maxwell.Hy, 0).simplify()],
             [Eqs[1].subs(maxwell.Ez, 0).doit().simplify(), Eqs[1].subs(maxwell.Hx, 0).simplify()],
             [Eqs[2].subs(maxwell.Hz, 0).doit().simplify(), Eqs[2].subs(maxwell.Ey, 0).simplify()],
             [Eqs[3].subs(maxwell.Hz, 0).doit().simplify(), Eqs[3].subs(maxwell.Ex, 0).simplify()],
             [Eqs[4].subs([(maxwell.Hx, 0), (maxwell.Hy, 0)]).doit().simplify(), Eqs[4].subs([(maxwell.Ez, 0), (maxwell.Hy, 0)]).subs(Derivative(0, maxwell.x), 0).simplify(), Eqs[4].subs([(maxwell.Ez, 0), (maxwell.Hx, 0)]).subs(Derivative(0, maxwell.y), 0).simplify()],
             [Eqs[5].subs([(maxwell.Ex, 0), (maxwell.Ey, 0)]).doit().simplify(), Eqs[5].subs([(maxwell.Hz, 0), (maxwell.Ey, 0)]).subs(Derivative(0, maxwell.x), 0).simplify(), Eqs[5].subs([(maxwell.Hz, 0), (maxwell.Ex, 0)]).subs(Derivative(0, maxwell.y), 0).simplify()]]
if vbose:
    for i in range(len(Eqs_b)):
        display(Math('Eqs_b[{}] = '.format(i) + latex(Matrix(Eqs_b[i]).T)))

if vbose:
    print('Eq1 boundary equations ')
Eqs1_b = [[Eqs_b[i][j].subs([(maxwell.mu, mu_0), (maxwell.eps, eps_0), (maxwell.sigma, 0.0), (maxwell.omega, omega_0), (k_0, k_0_val)]) for j in range(len(Eqs_b[i]))] for i in range(len(Eqs_b))]
if vbose:
    for i in range(len(Eqs1_b)):
        display(Math('EQs1_b[{}] = '.format(i) + latex(Matrix(Eqs1_b[i]).T)))

if vbose:
    print('Eq2 boundary equations ')
Eqs2_b = [[Eqs_b[i][j].subs([(maxwell.mu, mu_r*mu_0), (maxwell.eps, eps_r*eps_0), (maxwell.sigma, sig_graph/del_graph), (maxwell.omega, omega_0), (k_0, k_0_val)]) for j in range(len(Eqs_b[i]))] for i in range(len(Eqs_b))]
if vbose:
    for i in range(len(Eqs2_b)):
        display(Math('Eqs2_b[{}] = '.format(i) + latex(Matrix(Eqs2_b[i]).T)))

    
    
rg.AddExpression(region_fs, Eqs1, var_list)
rg.AddExpression(region_di, Eqs2, var_list)

eq_list1 = [{'zero':Float(0), 'one':(Eqs1_b[0][0]+Eqs1_b[0][1])/2.0, 'two':(Eqs1_b[0][0]+Eqs1_b[0][1])},
            {'zero':Float(0), 'one':(Eqs1_b[1][0]+Eqs1_b[1][1])/2.0, 'two':(Eqs1_b[1][0]+Eqs1_b[1][1])},
            {'zero':Float(0), 'one':(Eqs1_b[2][0]/2.0+Eqs1_b[2][1]), 'two':(Eqs1_b[2][0]+Eqs1_b[2][1])},
            {'zero':Float(0), 'one':(Eqs1_b[3][0]/2.0+Eqs1_b[3][1]), 'two':(Eqs1_b[3][0]+Eqs1_b[3][1])},
            {'zero':Float(0), 'one':(Eqs1_b[4][0]/4.0), 'two':(Eqs1_b[4][0]/2.0), 'two_x':(Eqs1_b[4][1]+Eqs1_b[4][2]/2.0), 'two_y':(Eqs1_b[4][1]/2.0+Eqs1_b[4][2]), 'three':(Eqs1_b[4][0]*3.0/4.0+Eqs1_b[4][1]+Eqs1_b[4][2]), 'four':(Eqs1_b[4][0]+Eqs1_b[4][1]+Eqs1_b[4][2])},
           {}]
var_type = ['SX', 'SY', 'SY', 'SX', 'N', 'C']
rg.SetRegionBoundaryEquations(region_fs, eq_list1, var_list, var_type)

eq_list2 = [{'zero':Float(0), 'one':(Eqs2_b[0][0]+Eqs2_b[0][1])/2.0, 'two':(Eqs2_b[0][0]+Eqs2_b[0][1])},
            {'zero':Float(0), 'one':(Eqs2_b[1][0]+Eqs2_b[1][1])/2.0, 'two':(Eqs2_b[1][0]+Eqs2_b[1][1])},
            {'zero':Float(0), 'one':(Eqs2_b[2][0]/2.0+Eqs2_b[2][1]), 'two':(Eqs2_b[2][0]+Eqs2_b[2][1])},
            {'zero':Float(0), 'one':(Eqs2_b[3][0]/2.0+Eqs2_b[3][1]), 'two':(Eqs2_b[3][0]+Eqs2_b[3][1])},
            {'zero':Float(0), 'one':(Eqs2_b[4][0]/4.0), 'two':(Eqs2_b[4][0]/2.0), 'two_x':(Eqs2_b[4][1]+Eqs1_b[4][2]/2.0), 'two_y':(Eqs2_b[4][1]/2.0+Eqs1_b[4][2]), 'three':(Eqs2_b[4][0]*3.0/4.0+Eqs2_b[4][1]+Eqs2_b[4][2]), 'four':(Eqs2_b[4][0]+Eqs2_b[4][1]+Eqs2_b[4][2])},
           {}]
var_type = ['SX', 'SY', 'SY', 'SX', 'N', 'C']
rg.SetRegionBoundaryEquations(region_di, eq_list2, var_list, var_type)


A_coo = rg.ConstructInitialMatrix(save=True)
if vbose:
    print("InitialMatrix nnz: ", A_coo.nnz)

A_coo = rg.ConstructInitialMatrix_Shared(A_coo, save=True)
if vbose:
    print("InitialMatrix_Shared nnz: ", A_coo.nnz)

A_coo = rg.ApplyDirichletBoundaryCondition(A_coo, save=True)

if vbose:
    print("InitialMatrixfinal nnz: ",A_coo.nnz) 

A_fin = rg.EliminateVarible_Direct(A_coo, [maxwell.Ez, maxwell.Hz])

k_eigs, k_vecs = rg.SolveEigenvalues(A_fin, 5, lambda_0=15.0)

print('k_eigs: ', k_eigs)

if vbose:
    print('k_eigs: ', k_eigs)
    print('k_vecs.shape: ', k_vecs.shape)


k_eigs:  [ 5.42497272-1.81351735j  5.42497272-1.81351735j  3.91856675-1.08699424j
  3.81398313-1.12728558j  3.63745319-1.2087989j ]


In [62]:
rg.GetFromSolutionIndexToInitialIndex(k_vecs[:,4], A_coo)


exy_cell = rg.InterpolateVariablesValuesToCellCenters([maxwell.Ex, maxwell.Ey])
ex = np.array(exy_cell[0])
ey = np.array(exy_cell[1])

ex_sq = rg.CalculateSurfaceIntegral(ex, np.conjugate(ex))
ey_sq = rg.CalculateSurfaceIntegral(ey, np.conjugate(ey))
exy_len = np.sqrt(ex_sq + ey_sq)
ex /= exy_len
ey /= exy_len
print('exy_len: ', exy_len)


exy_max = GetMaxComplexVec2D(ex, ey)

print(ex.shape, ey.shape)
print(exy_max.shape)

ex0_ex1 =  rg.CalculateSurfaceIntegral(ex, np.conjugate(ex_prev))
ey0_ey1 =  rg.CalculateSurfaceIntegral(ey, np.conjugate(ey_prev))

e0e1_corr = ex0_ex1 + ey0_ey1

print('correlation: ', abs(e0e1_corr))


exy_len:  (5.75616691887e-07+0j)
(3544,) (3544,)
(3544,)
correlation:  0.152877384062


In [35]:
cell_ref = rg.FindCellIndicesWithHighestValues([maxwell.Ex, maxwell.Ey], 2.0)
rg.RefineCells(cell_ref)


n_ratio:  47   val_max:  0.173262740383
n_high:  87
cells_in_extended:  [274, 518, 519, 603, 553, 601, 674, 790, 1271, 2437, 602, 673, 675, 1450, 1452, 1453, 1454, 980, 983, 2431, 2306, 772, 2309, 774, 2312, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621, 2622, 2623, 2624, 2625, 2629, 2631, 844, 845, 846, 1948, 1451, 2750, 979, 981, 982, 984, 2806, 2807, 2808, 2305, 2307, 2311, 2313, 4363, 4365, 4366, 4624, 4626, 4627, 4372, 4629, 4374, 4375, 4632, 2308, 2310, 4630, 4631, 4368, 4628, 3450, 3451, 3452, 3453, 3454, 3455, 3456, 3457, 3458, 3459, 3460, 3461, 3462, 3463, 3464, 3465, 3466, 3467, 3468, 4377, 1949, 1950, 1967, 1968, 4071, 4591, 4592, 4593, 4594, 4595, 4596]


stopping thread!
thread exiting..!
